<a href="https://colab.research.google.com/github/CRYPTO-Isaac/RansomwareDECT/blob/main/final_year_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>